Librerias a usar


In [1]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta

In [2]:
util.startLoop()

In [3]:
# Conexión a IB
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)  # Ajusta el clientId según tu configuración

<IB connected to 127.0.0.1:7497 clientId=1>

In [4]:
# Descargar datos de GBP/USD
symbol = Forex('GBPUSD')
end_time = datetime.now().strftime('%Y%m%d %H:%M:%S')
start_time = (datetime.now() - timedelta(days=30)).strftime('%Y%m%d %H:%M:%S')

data = ib.reqHistoricalData(
    symbol,
    endDateTime=end_time,
    durationStr='30 D',
    barSizeSetting='5 mins',
    whatToShow='MIDPOINT',
    useRTH=True
)

In [5]:
# Convertir a DataFrame
df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)


print(df.head())

                               open      high       low     close  volume  \
date                                                                        
2024-10-07 16:15:00-05:00  1.308300  1.308550  1.308220  1.308550    -1.0   
2024-10-07 16:20:00-05:00  1.308550  1.308560  1.308175  1.308175    -1.0   
2024-10-07 16:25:00-05:00  1.308175  1.308685  1.308175  1.308670    -1.0   
2024-10-07 16:30:00-05:00  1.308670  1.308720  1.308240  1.308240    -1.0   
2024-10-07 16:35:00-05:00  1.308240  1.308295  1.308115  1.308235    -1.0   

                           average  barCount  
date                                          
2024-10-07 16:15:00-05:00     -1.0        -1  
2024-10-07 16:20:00-05:00     -1.0        -1  
2024-10-07 16:25:00-05:00     -1.0        -1  
2024-10-07 16:30:00-05:00     -1.0        -1  
2024-10-07 16:35:00-05:00     -1.0        -1  


In [6]:
import requests

API_KEY = 'b35c56d955ee45178c703f7f79c1dfca'
url = f"https://newsapi.org/v2/everything?q=GBP%20USD%20economy&from={start_time}&sortBy=publishedAt&apiKey={API_KEY}"
response = requests.get(url)
news_data = response.json()

In [7]:
# Procesar noticias
articles = news_data.get('articles', [])
news_df = pd.DataFrame([{'title': a['title'], 'content': a['content'], 'publishedAt': a['publishedAt']} for a in articles])
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt'])
print(news_df.head())

                                               title  \
0  UK businesses drive growth and sustainability ...   
1  10 Influential Billionaires of 2024: Titans of...   
2  Dollar retains strength ahead of CPI, Fed spea...   
3  Western sanctions on Russia won’t work; they’r...   
4  Lunu Pay collaborates with Ingenico to bring c...   

                                             content               publishedAt  
0  UK businesses are playing a significant role i... 2024-11-13 07:54:23+00:00  
1  Legacy, innovation and influence define the wo... 2024-11-12 07:52:51+00:00  
2  Investing.com - The U.S. dollar rose Monday, c... 2024-11-11 09:26:21+00:00  
3  Efforts by the United States and its Western a... 2024-11-11 06:00:00+00:00  
4  Lunu Pay, a leader in Web3 payment solutions, ... 2024-11-07 15:51:00+00:00  


In [8]:
from ta import add_all_ta_features

df = add_all_ta_features(
    df,
    open="open",
    high="high",
    low="low",
    close="close",
    volume="volume",
    fillna=True
)
print(df.head())

c:\Users\alex_\AppData\Local\Programs\Python\Python312\Lib\site-packages\ta\trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


                               open      high       low     close  volume  \
date                                                                        
2024-10-07 16:15:00-05:00  1.308300  1.308550  1.308220  1.308550    -1.0   
2024-10-07 16:20:00-05:00  1.308550  1.308560  1.308175  1.308175    -1.0   
2024-10-07 16:25:00-05:00  1.308175  1.308685  1.308175  1.308670    -1.0   
2024-10-07 16:30:00-05:00  1.308670  1.308720  1.308240  1.308240    -1.0   
2024-10-07 16:35:00-05:00  1.308240  1.308295  1.308115  1.308235    -1.0   

                           average  barCount  volume_adi  volume_obv  \
date                                                                   
2024-10-07 16:15:00-05:00     -1.0        -1   -1.000000        -1.0   
2024-10-07 16:20:00-05:00     -1.0        -1    0.000000         0.0   
2024-10-07 16:25:00-05:00     -1.0        -1   -0.941176        -1.0   
2024-10-07 16:30:00-05:00     -1.0        -1    0.058824         0.0   
2024-10-07 16:35:00-05:00   

In [9]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis")
news_df['sentiment'] = news_df['content'].apply(lambda x: sentiment_analysis(x[:512])[0]['label'])
print(news_df.head())

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

c:\Users\alex_\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alex_\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: Failed to import transformers.models.distilbert.modeling_tf_distilbert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.